In [1]:
instrument = 'NIFTY'

In [2]:
import datetime
import numpy as np
import pandas as pd

In [3]:
from glob import glob

In [4]:
start_date = datetime.date(2023,1,1)
stop_date = datetime.date(2023,10,24)

In [5]:
files_path = f'..\\ML Data\\{instrument.lower()}'

In [6]:
all_files = glob(files_path+"\\*")
all_files_df = pd.DataFrame(all_files,columns=['path'])
all_files_df['date'] = all_files_df['path'].apply(lambda x: x.split('\\')[-1].split('.')[0])
all_files_df.sort_values('date',inplace=True)
all_files_df.reset_index(drop=True,inplace=True)

all_files_df['date'] = pd.to_datetime(all_files_df['date']).apply(lambda x: x.date())
all_files_df = all_files_df.set_index('date')

In [7]:
expiry_dates = {}

for date_idx, row in all_files_df.iterrows():

    data = pd.DataFrame()
    data = pd.read_parquet(row['path'])

    expiry_date = None
    expiry_date = data[data['expiry_date'] >= date_idx]['expiry_date'].min()
    
    expiry_dates[date_idx] = expiry_date

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
pip install fastparquet